In [ ]:
#StudentCourseData_Report10.csv -- Dataset

In [1029]:
import pandas as pd
import numpy as np
from mlxtend.preprocessing import TransactionEncoder
import dash
import dash_html_components as html
import dash_core_components as dcc
import dash.dependencies
from dash.dependencies import Input, Output, State
import dash_daq as daq
import dash_table as dt
import dash_bootstrap_components as dbc
import plotly
import base64
import plotly.graph_objs as go
import networkx as nx
import plotly.express as px
import dash_cytoscape as cyto
from plotly.subplots import make_subplots
import plotly.figure_factory as ff
from surprise import SVD, accuracy,Dataset
from surprise.reader import Reader
from surprise.model_selection import cross_validate,train_test_split
from surprise import SVD,SVDpp,SlopeOne,NMF,NormalPredictor,KNNBaseline,KNNBasic,KNNWithMeans,KNNWithZScore,BaselineOnly,CoClustering

In [78]:
df=pd.read_csv('StudentCourseData_Report (5).csv')

In [79]:
df.to_csv('studdata1.csv',header=False, index=False)

In [80]:
df_t=pd.read_csv('studdata1.csv')

In [81]:
df_t.head(3)

,Advanced Image Synthesis,Advanced Web Technologies,Cloud Web & Mobile,Cognitive Robot Systems,Computer Robot Vision,Computer Graphics,Gestaltung interaktiver Lehr-/Lern-Systeme,Development of Safe and Secure Software,Digital Games Research,Distributed Systems,...,Natural-Language-based Human-Computer Interaction,Neurocomputing and Organic Computing,Pattern and Component based Software Development,Peer-to-Peer Systems,Recommender Systems,Scientific Visualization,Test and Reliability of Digital Systems,Unnamed: 31,Unnamed: 32,Unnamed: 33
0,"Semester:2,Grade:B,Rating:2","Semester:1,Grade:B,Rating:2",NaN,NaN,NaN,NaN,NaN,"Semester:1,Grade:C,Rating:3",NaN,"Semester:1,Grade:C,Rating:3",...,NaN,NaN,NaN,"Semester:3,Grade:C,Rating:3",NaN,NaN,NaN,15-Jan-2020 02:53:09,NaN,anil4frds@gmail.com
1,NaN,"Semester:3,Grade:B,Rating:1","Semester:3,Grade:C,Rating:3","Semester:1,Grade:C,Rating:2",NaN,NaN,"Semester:4,Grade:B,Rating:4","Semester:2,Grade:B,Rating:3",NaN,"Semester:1,Grade:C,Rating:3",...,NaN,NaN,NaN,"Semester:2,Grade:B,Rating:3",NaN,NaN,NaN,14-Jan-2020 10:30:59,NaN,anil4frds@gmail.com
2,NaN,NaN,NaN,NaN,NaN,NaN,"Semester:2,Grade:D,Rating:1",NaN,NaN,"Semester:1,Grade:D,Rating:2",...,NaN,NaN,NaN,NaN,"Semester:2,Grade:D,Rating:2",NaN,NaN,14-Jan-2020 09:13:34,NaN,anil4frds@gmail.com


In [82]:
df_t.drop(['Unnamed: 31','Unnamed: 32','Unnamed: 33'],axis=1,inplace=True)

In [83]:
df_tt=df_t.reset_index()

In [84]:
dfrslt=(df_tt.set_index('index')
         .fillna('Semester,Grade,Rating')
         .stack()
         .str.split(',|;', expand=True)
         .stack()
         .str.split(':', expand=True)
         .reset_index(level=2, drop=True)
         .set_index(0, append=True)[1]
         .unstack()
         .rename_axis(('id','sub'))
         .rename_axis(None, axis=1)
         .reset_index()
         )

In [85]:
dfrslt=dfrslt.mask(dfrslt.isna(), np.nan)

In [86]:
dfrslt.rename(columns={'Semester':'semester','Grade':'grade','Rating':'rating'},inplace=True)

In [87]:
dfrslt.head(10)

,id,sub,grade,rating,semester
0,0,Advanced Image Synthesis,B,2,2
1,0,Advanced Web Technologies,B,2,1
2,0,Cloud Web & Mobile,NaN,NaN,NaN
3,0,Cognitive Robot Systems,NaN,NaN,NaN
4,0,Computer Graphics,NaN,NaN,NaN
5,0,Computer Robot Vision,NaN,NaN,NaN
6,0,Development of Safe and Secure Software,C,3,1
7,0,Digital Games Research,NaN,NaN,NaN
8,0,Distributed Systems,C,3,1
9,0,Electronic Communities and Social Networks,NaN,NaN,NaN


In [88]:
dfrslt_nan=dfrslt.dropna(thresh=3, axis=0)

In [89]:
df_subs1=dfrslt_nan.drop(['semester','grade','rating'],axis=1)

In [90]:
df_final=dfrslt_nan.assign(count=dfrslt_nan.groupby(['id','semester']).cumcount()).pivot_table(index = ['id','count'],columns = 'semester',
                        values = 'sub',aggfunc = ''.join).rename_axis(columns = None,index = [None,None])

In [91]:
df_final.head(40)

1  \
0  0                          Advanced Web Technologies   
   1            Development of Safe and Secure Software   
   2                                Distributed Systems   
1  0                            Cognitive Robot Systems   
   1                                Distributed Systems   
   2                                                NaN   
   3                                                NaN   
   4                                                NaN   
2  0                                Distributed Systems   
   1         Electronic Communities and Social Networks   
   2           Formal Specification of Software Systems   
   3                            Information Engineering   
   4                                 Information Mining   
3  0                                Distributed Systems   
   1         Electronic Communities and Social Networks   
   2                                                NaN   
4  0  Internet of Things- Protocols and System Software   
   1                                                NaN   
5  0                                 Cloud Web & Mobile   
   1                              Computer Robot Vision   
   2                                Distributed Systems   
   3           Formal Specification of Software Systems   
   4                                                NaN   
6  0         Electronic Communities and Social Networks   
   1           Formal Specification of Software Systems   
   2                                 Learning Analytics   
7  0                                Recommender Systems   
8  0                           Advanced Image Synthesis   
   1                            Cognitive Robot Systems   
   2                              Computer Robot Vision   
   3                           Scientific Visualization   
   4                                                NaN   
9  0                             Digital Games Research   
   1                                Distributed Systems   
   2         Electronic Communities and Social Networks   
   3                                 Learning Analytics   
10 0                              Computer Robot Vision   
   1         Electronic Communities and Social Networks   
   2           Formal Specification of Software Systems   
11 0                                Distributed Systems   

                                                      2  \
0  0                           Advanced Image Synthesis   
   1                                 Learning Analytics   
   2                                                NaN   
1  0            Development of Safe and Secure Software   
   1                              Information Retrieval   
   2                               Peer-to-Peer Systems   
   3                                                NaN   
   4                                                NaN   
2  0         Gestaltung interaktiver Lehr-/Lern-Systeme   
   1                              Information Retrieval   
   2                                Interaktive Systeme   
   3  Internet of Things- Protocols and System Software   
   4                                Recommender Systems   
3  0         Gestaltung interaktiver Lehr-/Lern-Systeme   
   1                                Interaktive Systeme   
   2                               Peer-to-Peer Systems   
4  0                                Distributed Systems   
   1            Test and Reliability of Digital Systems   
5  0            Development of Safe and Secure Software   
   1                                Interaktive Systeme   
   2  Internet of Things- Protocols and System Software   
   3   Pattern and Component based Software Development   
   4                               Peer-to-Peer Systems   
6  0                          Advanced Web Technologies   
   1                                Interaktive Systeme   
   2                                                NaN   
7  0                                     

In [92]:
df_final.to_csv('category1.csv')

In [93]:
df_processed=df_subs1.groupby('id').sub.apply(list).reset_index()

In [94]:
df_processed.head(5)

,id,sub
0,0,"[Advanced Image Synthesis, Advanced Web Techno..."
1,1,"[Advanced Web Technologies, Cloud Web & Mobile..."
2,2,"[Distributed Systems, Electronic Communities a..."
3,3,"[Distributed Systems, Electronic Communities a..."
4,4,"[Distributed Systems, Internet of Things- Prot..."


In [95]:
df_processed.set_index('id',inplace=True)

In [96]:
dataset=list(df_processed['sub'].values)

In [97]:
te = TransactionEncoder()
te_ary = te.fit(dataset).transform(dataset)
df_encoded = pd.DataFrame(te_ary, columns=te.columns_)
df_encoded

,Advanced Image Synthesis,Advanced Web Technologies,Cloud Web & Mobile,Cognitive Robot Systems,Computer Graphics,Computer Robot Vision,Development of Safe and Secure Software,Digital Games Research,Distributed Systems,Electronic Communities and Social Networks,...,Interaktive Systeme,Internet of Things- Protocols and System Software,Learning Analytics,Natural-Language-based Human-Computer Interaction,Neurocomputing and Organic Computing,Pattern and Component based Software Development,Peer-to-Peer Systems,Recommender Systems,Scientific Visualization,Test and Reliability of Digital Systems
0,True,True,False,False,False,False,True,False,True,False,...,False,False,True,False,False,False,True,False,False,False
1,False,True,True,True,False,False,True,False,True,True,...,False,False,True,False,False,False,True,False,False,False
2,False,False,False,False,False,False,False,False,True,True,...,True,True,True,False,False,False,False,True,False,False
3,False,False,False,False,False,False,False,False,True,True,...,True,False,False,False,False,False,True,False,False,False
4,False,False,False,False,False,False,False,False,True,False,...,False,True,False,False,False,False,True,False,False,True
5,False,True,True,False,False,True,True,False,True,False,...,True,True,False,False,False,True,True,False,False,False
6,False,True,False,False,False,False,False,False,False,True,...,True,False,True,False,False,False,False,False,False,False
7,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,True,False,False
8,True,True,False,True,True,True,True,False,True,False,...,False,True,False,False,False,False,True,False,True,False
9,False,True,False,False,False,False,False,True,True,True,...,False,False,True,False,False,False,False,False,False,False


In [98]:
from mlxtend.frequent_patterns import apriori

apriori(df_encoded, min_support=0.45,use_colnames=True)

,support,itemsets
0,0.451613,(Advanced Web Technologies)
1,0.548387,(Distributed Systems)
2,0.451613,(Information Retrieval)


In [99]:
frequent_itemsets = apriori(df_encoded, min_support=0.05, use_colnames=True,verbose=1)
frequent_itemsets['length'] = frequent_itemsets['itemsets'].apply(lambda x: len(x))
frequent_itemsets

Processing 10 combinations | Sampling itemset size 10


,support,itemsets,length
0,0.129032,(Advanced Image Synthesis),1
1,0.451613,(Advanced Web Technologies),1
2,0.225806,(Cloud Web & Mobile),1
3,0.129032,(Cognitive Robot Systems),1
4,0.129032,(Computer Graphics),1
5,0.161290,(Computer Robot Vision),1
6,0.225806,(Development of Safe and Secure Software),1
7,0.064516,(Digital Games Research),1
8,0.548387,(Distributed Systems),1
9,0.419355,(Electronic Communities and Social Networks),1


In [100]:
from mlxtend.frequent_patterns import association_rules

rules=association_rules(frequent_itemsets,metric="confidence",min_threshold=0.1)

In [101]:
rules.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(Advanced Web Technologies),(Advanced Image Synthesis),0.451613,0.129032,0.096774,0.214286,1.660714,0.038502,1.108504
1,(Advanced Image Synthesis),(Advanced Web Technologies),0.129032,0.451613,0.096774,0.750000,1.660714,0.038502,2.193548
2,(Advanced Image Synthesis),(Cognitive Robot Systems),0.129032,0.129032,0.064516,0.500000,3.875000,0.047867,1.741935
3,(Cognitive Robot Systems),(Advanced Image Synthesis),0.129032,0.129032,0.064516,0.500000,3.875000,0.047867,1.741935
4,(Advanced Image Synthesis),(Computer Graphics),0.129032,0.129032,0.064516,0.500000,3.875000,0.047867,1.741935


In [102]:
rules.shape

(68160, 9)

In [103]:
l=[]
for i,j in zip(range(0,len(rules.iloc[:,0])),range(0,len(rules.iloc[:,1]))):
    itemset = set(rules.iloc[i,0]) | set(rules.iloc[i,1])
    l.append(tuple(itemset))

In [104]:
len(l)

68160

In [105]:
import functools
def reduce_concat(x, sep=""):
    return functools.reduce(lambda x, y: str(x) + sep + str(y), x)

def paste(*lists, sep=" ", collapse=None):
    result = map(lambda x: reduce_concat(x, sep=sep), zip(*lists))
    if collapse is not None:
        return reduce_concat(result, sep=collapse)
    return list(result)

In [106]:
antecedents=[]
for i in range(0,len(rules.iloc[:,0])):
    antecedents.append(list(rules.iloc[i,0]))

In [107]:
consequents=[]
for i in range(0,len(rules.iloc[:,1])):
    consequents.append(list(rules.iloc[i,1]))

In [108]:
list_join=paste(antecedents, consequents, sep=',')

In [109]:
df_join=pd.DataFrame(list_join)

In [110]:
halfjoins=df_join[0].str.replace('[','')

In [111]:
ohnequote=halfjoins.str.replace(']','')

In [112]:
finalentry=ohnequote.str.replace("'","")

In [113]:
newrules=rules.assign(itemsets=finalentry)

In [485]:
max_support=newrules_up[newrules_up['support']==max(newrules_up['support'])]

In [486]:
df_max_support=max_support[['support','confidence','itemsets1']]

In [487]:
df_max_support.head(10)

,support,confidence,itemsets1
154,0.290323,0.692308,"(Electronic Communities and Social Networks,Di..."
155,0.290323,0.529412,"(Distributed Systems,Electronic Communities an..."
164,0.290323,0.642857,"(Information Retrieval,Distributed Systems,)"
165,0.290323,0.529412,"(Distributed Systems,Information Retrieval,)"
168,0.290323,0.750000,(Internet of Things- Protocols and System Soft...
169,0.290323,0.529412,"(Distributed Systems,Internet of Things- Proto..."
174,0.290323,0.818182,"(Peer-to-Peer Systems,Distributed Systems,)"
175,0.290323,0.529412,"(Distributed Systems,Peer-to-Peer Systems,)"
236,0.290323,0.900000,"(Information Engineering,Information Retrieval,)"
237,0.290323,0.642857,"(Information Retrieval,Information Engineering,)"


In [488]:
##itemsets with high confidence
table_freqsets = ff.create_table(df_max_support)

In [115]:
l1=[]
for val in newrules['itemsets']:
    l1.append((val,))

In [116]:
newrules_up=newrules.assign(itemsets1=l1,inplace=True)

In [117]:
newrules_up.drop(['itemsets'],axis=1,inplace=True)

In [118]:
newrules_up.head(2)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,itemsets1,inplace
0,(Advanced Web Technologies),(Advanced Image Synthesis),0.451613,0.129032,0.096774,0.214286,1.660714,0.038502,1.108504,"(Advanced Web Technologies,Advanced Image Synt...",True
1,(Advanced Image Synthesis),(Advanced Web Technologies),0.129032,0.451613,0.096774,0.750000,1.660714,0.038502,2.193548,"(Advanced Image Synthesis,Advanced Web Technol...",True


In [125]:
li_suco=[]

In [126]:
df_rec=dfrslt_nan

In [127]:
s=df_rec['sub'].astype('category')

In [128]:
grade=df_rec['grade'].astype('category')

In [129]:
df_rec=df_rec.fillna(0)

In [130]:
df_rec['rating']=df_rec['rating'].astype(int)

In [131]:
df_rec['course id']=s.cat.codes

In [132]:
df_rec['grade_codes']=grade.cat.codes

In [133]:
df_rec[['grade','grade_codes']]

,grade,grade_codes
0,B,1
1,B,1
6,C,2
8,C,2
16,B,1
20,B,1
27,C,2
32,B,1
33,C,2
34,C,2


In [134]:
df_useritem=df_rec[['id','course id','sub','rating']]

In [135]:
df_useritem.dtypes

id            int64
course id      int8
sub          object
rating        int32
dtype: object

In [136]:
#df_useritem[['rating']]=df_useritem[['rating']].apply(pd.to_numeric) 

In [137]:
#df_rec[['rating']]=df_rec[['rating']].apply(pd.to_numeric) 

## Recommendation System

In [138]:
reader=Reader(rating_scale=(1, 5))

In [139]:
data = Dataset.load_from_df(df_useritem[['id', 'course id', 'rating']],reader)

In [140]:
df_subid=df_useritem[['sub','course id']]

In [141]:
df_subid.drop_duplicates()

,sub,course id
0,Advanced Image Synthesis,0
1,Advanced Web Technologies,1
6,Development of Safe and Secure Software,6
8,Distributed Systems,8
16,Information Retrieval,14
20,Learning Analytics,17
27,Peer-to-Peer Systems,21
33,Cloud Web & Mobile,2
34,Cognitive Robot Systems,3
40,Electronic Communities and Social Networks,9


In [142]:
df_subid.sort_values(by=['course id'], inplace=True)

In [143]:
df_subid.drop_duplicates(inplace=True)

In [144]:
df_subid.head(5)

,sub,course id
0,Advanced Image Synthesis,0
652,Advanced Web Technologies,1
157,Cloud Web & Mobile,2
251,Cognitive Robot Systems,3
903,Computer Graphics,4


In [147]:
#cross_validate(algo, data, measures=['RMSE'], cv=3, verbose=False)

In [148]:
trainset, testset = train_test_split(data, test_size=0.2)

In [149]:
benchmark = []
# Iterate over all algorithms
for algorithm in [SVD(), SVDpp(), SlopeOne(), NMF(), NormalPredictor(), KNNBaseline(), KNNBasic(), KNNWithMeans(), KNNWithZScore(), BaselineOnly(), CoClustering()]:
    # Perform cross validation
    try:
        results = cross_validate(algorithm, data, measures=['RMSE'], cv=2, verbose=True)
    except ZeroDivisionError:
        print('Nan values are eliminated')
    
    # Get results & append algorithm name
    tmp = pd.DataFrame.from_dict(results).mean(axis=0)
    tmp = tmp.append(pd.Series([str(algorithm).split(' ')[0].split('.')[-1]], index=['Algorithm']))
    benchmark.append(tmp)
    
pd.DataFrame(benchmark).set_index('Algorithm').sort_values('test_rmse')

Evaluating RMSE of algorithm SVD on 2 split(s).

                  Fold 1  Fold 2  Mean    Std     
RMSE (testset)    1.2076  1.3312  1.2694  0.0618  
Fit time          0.01    0.01    0.01    0.00    
Test time         0.00    0.00    0.00    0.00    
Evaluating RMSE of algorithm SVDpp on 2 split(s).

                  Fold 1  Fold 2  Mean    Std     
RMSE (testset)    1.2482  1.3159  1.2820  0.0338  
Fit time          0.02    0.02    0.02    0.00    
Test time         0.00    0.00    0.00    0.00    
Evaluating RMSE of algorithm SlopeOne on 2 split(s).

                  Fold 1  Fold 2  Mean    Std     
RMSE (testset)    1.5201  1.5486  1.5344  0.0143  
Fit time          0.00    0.00    0.00    0.00    
Test time         0.00    0.00    0.00    0.00    
Nan values are eliminated
Evaluating RMSE of algorithm NormalPredictor on 2 split(s).

                  Fold 1  Fold 2  Mean    Std     
RMSE (testset)    1.9077  2.0677  1.9877  0.0800  
Fit time          0.00    0.00    0.00    0.0

,test_rmse,fit_time,test_time
Algorithm,,,
KNNWithMeans,1.257783,0.000996,0.002013
SVD,1.269444,0.007978,0.001495
SVDpp,1.282023,0.022458,0.001996
BaselineOnly,1.336863,0.000997,0.000000
KNNWithZScore,1.487829,0.001495,0.001496
KNNBaseline,1.515321,0.001494,0.000999
KNNBasic,1.521155,0.000000,0.001513
SlopeOne,1.534358,0.000499,0.000979
NMF,1.534358,0.000499,0.000979


In [150]:
sim_options = {
    "name": "msd",
    "user_based": True,
}




In [151]:

from surprise.model_selection import GridSearchCV
algo = KNNWithMeans(sim_options=sim_options)
#gs.fit(data)

In [152]:
algo.fit(trainset)

Computing the msd similarity matrix...
Done computing similarity matrix.


In [153]:
predictions=algo.test(testset)

In [154]:
accuracy.rmse(predictions)

RMSE: 1.2590


1.2589547236766523

In [155]:
df_rec[df_rec['sub']=='Knowledge-Based Systems']

,id,sub,grade,rating,semester,course id,grade_codes


In [156]:
#list_subs=['Interaktive Systeme','Information Mining','Advanced Web Technologies']

In [158]:
list_neighbors=algo.get_neighbors(18,k=3)

In [159]:
list_neighbors

[3, 5, 2]

In [160]:
def get_neighbors(list1,k):
    list_neighbors=[]
    list_subs_up=[]
    for val in list1:
        list_subs_up.append(int(df_subid[df_subid['sub']==val]['course id'].values))  
    for val in list_subs_up:
        list_neighbors.append(algo.get_neighbors(val,k))
        print("The students who chose these subjects chose the following subjects:",algo.get_neighbors(val,k))
    return list_neighbors

In [161]:
def get_subs(list1):
    l_neigh=[]
    for val in list1:
        l_neigh.append(((df_useritem[df_useritem['course id']==val]['sub']).drop_duplicates()).tolist())
        list_neigh = [item for sublist in l_neigh for item in sublist]
    return list_neigh

In [162]:
#l_up=get_neighbors(list_subs,3)

NameError: name 'list_subs' is not defined

In [163]:
def recommend_courses(list_subs):
    recommendation_list=[]
    for val in get_neighbors(list_subs,3):
        recommendation_list.append(get_subs(val))
    return recommendation_list

In [606]:
recommend_courses(['Advanced Web Technologies']),recommend_courses(['Information Mining']),recommend_courses(['Information Engineering']),recommend_courses(['Gestaltung interaktiver Lehr-/Lern-Systeme']),recommend_courses(['Test and Reliability of Digital Systems']),recommend_courses(['Fault Diagnosis and Fault Tolerance in Technical Systems'])

The students who chose these subjects chose the following subjects: [12, 21, 7]
The students who chose these subjects chose the following subjects: [5, 14, 19]
The students who chose these subjects chose the following subjects: [0, 1, 17]
The students who chose these subjects chose the following subjects: [6, 19, 1]
The students who chose these subjects chose the following subjects: [15, 6, 8]


TypeError: only size-1 arrays can be converted to Python scalars

In [166]:
df_sankey_daig=pd.read_csv('category1.csv')

In [167]:
df_sankey_daig.drop(['Unnamed: 0'],axis=1,inplace=True)

In [168]:
df_rec[df_rec['sub']=='Advanced Image Synthesis']

,id,sub,grade,rating,semester,course id,grade_codes
0,0,Advanced Image Synthesis,B,2,2,0,1
248,8,Advanced Image Synthesis,F,0,1,0,5
682,22,Advanced Image Synthesis,C,1,2,0,2
744,24,Advanced Image Synthesis,0,0,Other,0,-1


In [169]:
df_ratings=df_rec.groupby('sub').rating.mean().reset_index()

In [170]:
df_ratings.head(4)

,sub,rating
0,Advanced Image Synthesis,0.750000
1,Advanced Web Technologies,1.357143
2,Cloud Web & Mobile,1.428571
3,Cognitive Robot Systems,2.000000


## Dash

In [171]:
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

In [172]:
fig3 = px.scatter(newrules_up, x="support", y="confidence", size="lift",hover_name="itemsets1")

In [473]:
#fig5=px.histogram(df_subs1,x='sub',color='sub')

In [476]:
fig5 = go.Figure(data=[go.Histogram(y=df_subs1['sub'].values)])
fig5.show()

In [177]:
df_rec.dtypes

id              int64
sub            object
grade          object
rating          int32
semester       object
course id        int8
grade_codes      int8
dtype: object

In [178]:
df_rec1=df_rec

In [179]:
df_rec1['semester'].unique()[5]

0

In [180]:
df_rec1[df_rec1['semester']==0]=df_rec1[df_rec1['semester']==0].applymap(str).iloc[0,3]

In [181]:
df_rec1['semester'].unique()

array(['2', '1', '3', '4', 'Other', '0'], dtype=object)

In [182]:
df_rec1=df_rec1[df_rec1['rating']!='0']

In [183]:
df_rec1['rating'].unique()

array([2, 3, 1, 4, 5, 0], dtype=object)

In [899]:
fig6=px.scatter(df_rec1.sort_values(by=['semester']), x='course id', y='rating',color='sub',animation_frame="semester", animation_group="course id")
#px.scatter(df_rec, x="sub", y="rating", animation_frame="rating", animation_group="course id",size='rating',
            #color="course id", hover_name="sub")
fig6.update_layout(title="Ratings semester wise",width=900)
fig6.show()

In [257]:
df_sankey_daig[df_sankey_daig['1']=='Development of Safe and Secure Software']

,Unnamed: 1,1,2,3,4,Other
1,1,Development of Safe and Secure Software,Learning Analytics,Peer-to-Peer Systems,NaN,NaN
48,0,Development of Safe and Secure Software,NaN,NaN,NaN,NaN
90,0,Development of Safe and Secure Software,Gestaltung interaktiver Lehr-/Lern-Systeme,Advanced Web Technologies,NaN,NaN


In [189]:
fig7 = px.parallel_categories(df_sankey_daig[10:50], dimensions=['1', '2', '3','4','Other'],
                color='Unnamed: 1', color_continuous_scale=px.colors.sequential.Inferno,
                labels={'1':'Semester1', '2':'Semester2', '3':'Semester3','4':'Semester4','Other':'OtherSemesters'},title='Student Behavioral Flow from Semester to Semester',)

In [1013]:
df_sankey_small=df_sankey_daig[1:50]
categorical_dimensions = ['1', '2', '3','4','Other']

dimensions = [dict(values=df_sankey_daig[label], label=label) for label in categorical_dimensions]

# Build colorscale
# [aliceblue, antiquewhite, aqua, aquamarine, azure,
#             beige, bisque, black, blanchedalmond, blue,
#             blueviolet, brown, burlywood, cadetblue,
#             chartreuse, chocolate, coral, cornflowerblue,
#             cornsilk, crimson, cyan, darkblue, darkcyan,
#             darkgoldenrod, darkgray, darkgrey, darkgreen,
#             darkkhaki, darkmagenta, darkolivegreen, darkorange,
#             darkorchid, darkred, darksalmon, darkseagreen,
#             darkslateblue, darkslategray, darkslategrey,
#             darkturquoise, darkviolet, deeppink, deepskyblue,
#             dimgray, dimgrey, dodgerblue, firebrick,
#             floralwhite, forestgreen, fuchsia, gainsboro,
#             ghostwhite, gold, goldenrod, gray, grey, green,
#             greenyellow, honeydew, hotpink, indianred, indigo,
#             ivory, khaki, lavender, lavenderblush, lawngreen,
#             lemonchiffon, lightblue, lightcoral, lightcyan,
#             lightgoldenrodyellow, lightgray, lightgrey,
#             lightgreen, lightpink, lightsalmon, lightseagreen,
#             lightskyblue, lightslategray, lightslategrey,
#             lightsteelblue, lightyellow, lime, limegreen,
#             linen, magenta, maroon, mediumaquamarine,
#             mediumblue, mediumorchid, mediumpurple,
#             mediumseagreen, mediumslateblue, mediumspringgreen,
#             mediumturquoise, mediumvioletred, midnightblue,
#             mintcream, mistyrose, moccasin, navajowhite, navy,
#             oldlace, olive, olivedrab, orange, orangered,
#             orchid, palegoldenrod, palegreen, paleturquoise,
#             palevioletred, papayawhip, peachpuff, peru, pink,
#             plum, powderblue, purple, red, rosybrown,
#             royalblue, rebeccapurple, saddlebrown, salmon,
#             sandybrown, seagreen, seashell, sienna, silver,
#             skyblue, slateblue, slategray, slategrey, snow,
#             springgreen, steelblue, tan, teal, thistle, tomato,
#             turquoise, violet, wheat, white, whitesmoke,
#             yellow, yellowgreen
#  ]
color = ['peru','thistle','indianred','wheat','violet','salmon','tan','palegoldenrod','orchid','seagreen','navajowhite','mediumseagreen','papayawhip',
        'khaki','grey','honeydew','lemonchiffon','moccasin','darkolivegreen','papayawhip','powderblue','indigo','beige']
#colorscale = [[0, 'lightsteelblue'], [1, 'mediumseagreen']];

# Build figure as FigureWidget
fig_p = go.FigureWidget(
    data=[go.Parcats(
        domain={'x': [0.2, 1],'y': [0, 1]}, dimensions=dimensions,
        line={ 'cmin': 0,
              'cmax': 1, 'color': color, 'shape': 'hspline'})
    ])


In [1014]:
fig_p.update_layout(height=500,width=1600,title='Student behavioral flow from Semester to Semester')

FigureWidget({
    'data': [{'dimensions': [{'label': '1',
                              'values': array(['Adv…

## Plots based on Subject

In [335]:
fig_subplot1 = make_subplots(
    rows=1, cols=3,
    subplot_titles=("Plot 1", "Plot 2", "Plot 3"))

In [336]:
series_ratings=df_rec.groupby('sub').rating.value_counts(sort=True).to_frame()
ratings_graph_data=series_ratings.rename(columns={series_ratings.columns[0]: 'frequency'}).reset_index()
#pd.DataFrame(series_ratings)

In [337]:
series_grades=df_rec.groupby('sub').grade.value_counts(sort=True).to_frame()
grade_graph_data=series_grades.rename(columns={series_grades.columns[0]: 'frequency'}).reset_index()
#pd.DataFrame(series_ratings)

### Count for recommending grades

In [711]:
grade_A=grade_graph_data[grade_graph_data['grade']=='A']
grade_F=grade_graph_data[grade_graph_data['grade']=='F']

In [716]:
easy_sub=grade_A[grade_A['frequency']==max(grade_A['frequency'])]['sub'].values[0]

In [717]:
diff_sub=grade_F[grade_F['frequency']==max(grade_F['frequency'])]['sub'].values[0]

In [338]:
series_semester=df_rec.groupby('sub').semester.value_counts(sort=True).to_frame()
semester_graph_data=series_semester.rename(columns={series_semester.columns[0]: 'frequency'}).reset_index()
#pd.DataFrame(series_ratings)

In [339]:
def get_rating_frequency(value):
    return ratings_graph_data[ratings_graph_data['sub']==value]

In [340]:
def get_grade_frequency(value):
    return grade_graph_data[grade_graph_data['sub']==value]

In [341]:
def get_semester_frequency(value):
    return semester_graph_data[semester_graph_data['sub']==value]

In [758]:
fig11=go.Figure([go.Bar(x=list(df_ratings['sub'].values),y=list(df_ratings['rating'].values))])
fig11.update_layout(height=600,width=900,font=dict(size=7))
fig12=go.Figure([go.Bar(x=list(df_rec['sub'].values),y=list(df_rec['semester'].values))])
fig12.update_layout(height=600,width=900)
fig13=go.Figure([go.Bar(x=["Advanced Web Technologies","Distributed Systems","Social Networks"],y=[1, 3, 4])])

In [343]:
fig11.show()

## Semester Wise

In [344]:
sem_data=df_rec.groupby('semester').sub.value_counts(sort=True).to_frame()
semester_data=sem_data.rename(columns={sem_data.columns[0]: 'frequency'}).reset_index()

In [784]:
sem11=list(semester_data[semester_data['semester']=='1']['sub'].values)
sem12=list(semester_data[semester_data['semester']=='1']['frequency'].values)
sem21=list(semester_data[semester_data['semester']=='2']['sub'].values)
sem22=list(semester_data[semester_data['semester']=='2']['frequency'].values)
sem31=list(semester_data[semester_data['semester']=='3']['sub'].values)
sem32=list(semester_data[semester_data['semester']=='3']['frequency'].values)
sem41=list(semester_data[semester_data['semester']=='4']['sub'].values)
sem42=list(semester_data[semester_data['semester']=='4']['frequency'].values)
semo1=list(semester_data[semester_data['semester']=='Other']['sub'].values)
semo2=list(semester_data[semester_data['semester']=='Other']['frequency'].values)

## Grade Wise

In [346]:
grade_data=df_rec.groupby('grade').sub.value_counts(sort=True).to_frame()
grade_graph_data=grade_data.rename(columns={grade_data.columns[0]: 'frequency'}).reset_index()

In [347]:
gradea1=list(grade_graph_data[grade_graph_data['grade']=='A']['sub'].values)
gradea2=list(grade_graph_data[grade_graph_data['grade']=='A']['frequency'].values)
gradeb1=list(grade_graph_data[grade_graph_data['grade']=='B']['sub'].values)
gradeb2=list(grade_graph_data[grade_graph_data['grade']=='B']['frequency'].values)
gradec1=list(grade_graph_data[grade_graph_data['grade']=='C']['sub'].values)
gradec2=list(grade_graph_data[grade_graph_data['grade']=='C']['frequency'].values)
graded1=list(grade_graph_data[grade_graph_data['grade']=='D']['sub'].values)
graded2=list(grade_graph_data[grade_graph_data['grade']=='D']['frequency'].values)
gradee1=list(grade_graph_data[grade_graph_data['grade']=='E']['sub'].values)
gradee2=list(grade_graph_data[grade_graph_data['grade']=='E']['frequency'].values)
gradef1=list(grade_graph_data[grade_graph_data['grade']=='F']['sub'].values)
gradef2=list(grade_graph_data[grade_graph_data['grade']=='F']['frequency'].values)

In [348]:
df_rec[df_rec['grade'] == 0]

,id,sub,grade,rating,semester,course id,grade_codes
82,2,Learning Analytics,0,1,3,17,-1
442,14,Distributed Systems,0,2,2,8,-1
469,15,Computer Graphics,0,1,1,4,-1
473,15,Distributed Systems,0,2,1,8,-1
480,15,Information Mining,0,3,1,13,-1
485,15,Learning Analytics,0,1,1,17,-1
744,24,Advanced Image Synthesis,0,0,Other,0,-1


In [914]:
df_filtered = df_rec[df_rec['semester'] != 'Other' ] 

In [915]:
df_filtered=df_filtered[df_filtered['grade']!=0]

In [916]:
df_filtered=df_filtered[df_filtered['grade']!='F']

In [917]:
df_filtered1 =df_rec[(df_rec['grade'] != 0 ) & (df_rec['grade'] != 'F' )] 

In [918]:
df_filtered.groupby(['id','semester']).sub.value_counts()

id  semester  sub                                              
0   1         Advanced Web Technologies                            1
              Development of Safe and Secure Software              1
              Distributed Systems                                  1
    2         Advanced Image Synthesis                             1
              Learning Analytics                                   1
    3         Information Retrieval                                1
              Peer-to-Peer Systems                                 1
1   1         Cognitive Robot Systems                              1
              Distributed Systems                                  1
    2         Development of Safe and Secure Software              1
              Information Retrieval                                1
              Peer-to-Peer Systems                                 1
    3         Advanced Web Technologies                            1
              Cloud Web & Mobile       

In [919]:
df_filtered.groupby('id')['sub'].value_counts(sort=True).to_frame()

sub
id sub                                                   
0  Advanced Image Synthesis                             1
   Advanced Web Technologies                            1
   Development of Safe and Secure Software              1
   Distributed Systems                                  1
   Information Retrieval                                1
   Learning Analytics                                   1
   Peer-to-Peer Systems                                 1
1  Advanced Web Technologies                            1
   Cloud Web & Mobile                                   1
   Cognitive Robot Systems                              1
   Development of Safe and Secure Software              1
   Distributed Systems                                  1
   Electronic Communities and Social Networks           1
   Gestaltung interaktiver Lehr-/Lern-Systeme           1
   Information Engineering                              1
   Information Retrieval                                1
   Learning Analytics                                   1
   Peer-to-Peer Systems                                 1
2  Distributed Systems                                  1
   Electronic Communities and Social Networks           1
   Formal Specification of Software Systems             1
   Gestaltung interaktiver Lehr-/Lern-Systeme           1
   Information Engineering                              1
   Information Mining                                   1
   Information Retrieval                                1
   Interaktive Systeme                                  1
   Internet of Things- Protocols and System Software    1
   Recommender Systems                                  1
3  Distributed Systems                                  1
   Electronic Communities and Social Networks           1
...                                                   ...
24 Internet of Things- Protocols and System Software    1
25 Cloud Web & Mobile                                   1
   Information Mining                                   1
   Natural-Language-based Human-Computer Interaction    1
26 Advanced Web Technologies                            1
   Gestaltung interaktiver Lehr-/Lern-Systeme           1
   Information Retrieval                                1
   Internet of Things- Protocols and System Software    1
27 Advanced Web Technologies                            1
28 Advanced Web Technologies                            1
   Peer-to-Peer Systems                                 1
29 Distributed Systems                                  1
   Electronic Communities and Social Networks           1
   Formal Specification of Software Systems             1
   Gestaltung interaktiver Lehr-/Lern-Systeme           1
   Information Engineering                              1
   Information Mining                                   1
   Information Retrieval                                1
   Interaktive Systeme                                  1
   Internet of Things- Protocols and System Software    1
   Recommender Systems                                  1
30 Cloud Web & Mobile                                   1
   Distributed Systems                                  1
   Electronic Communities and Social Networks           1
   Information Engineering                              1
   Information Mining                                   1
   Information Retrieval                                1
   Internet of Things- Protocols and System Software    1
   Peer-to-Peer Systems                                 1
0  0                                                    1

[157 rows x 1 columns]

In [920]:
freq_prosem=df_filtered1.groupby(['id','semester']).sub.value_counts(sort=True).to_frame()

In [921]:
#fre_prosem_graph=freq_prosem.rename(columns={freq_prosem.columns[0]: 'frequency'}).reset_index()

In [922]:
#sum_data=fre_prosem_graph.groupby(['id','semester']).frequency.sum(sort=True).to_frame()
#sum_data_graph=sum_data.rename(columns={sum_data.columns[0]: 'sum'}).reset_index()

In [923]:
#countd1=sum_data_graph.groupby(['semester','sum']).id.value_counts(sort=True).to_frame()
#countd1_graph=countd1.rename(columns={countd1.columns[0]: 'frequency'}).reset_index()

In [924]:
#countd2=countd1_graph.groupby(['semester','sum']).frequency.sum(sort=True).to_frame()
#countd2_graph=countd2.rename(columns={countd2.columns[0]: 'frequency'}).reset_index()

In [925]:
#countd2_graph[countd2_graph['semester']=='Other']

In [926]:
sem1=countd2_graph[countd2_graph['semester']=='1']
sem2=countd2_graph[countd2_graph['semester']=='2']
sem3=countd2_graph[countd2_graph['semester']=='3']
sem4=countd2_graph[countd2_graph['semester']=='4']
semo=countd2_graph[countd2_graph['semester']=='Other']

In [927]:
sem1.head(4)

,semester,sum,frequency
1,1,1,5
2,1,2,10
3,1,3,6
4,1,4,1


#### Frequency of students for the no.of courses

In [928]:
# fig_semfreq = go.Figure(data=[
#     go.Bar(name='Semester 1', x=list(sem1['sum'].values), y=list(sem1['frequency'].values)),
#     go.Bar(name='Semester 2', x=list(sem2['sum'].values), y=list(sem2['frequency'].values)),
#     go.Bar(name='Semester 3', x=list(sem3['sum'].values), y=list(sem3['frequency'].values)),
#     go.Bar(name='Semester 4', x=list(sem4['sum'].values), y=list(sem4['frequency'].values)),
#     go.Bar(name='Other Semester', x=list(semo['sum'].values), y=list(semo['frequency'].values)),
# ])
# fig_semfreq.update_layout(barmode='group',yaxis=dict(
#         title='No.of students',
#         titlefont_size=16,
#         tickfont_size=14,
#     ),xaxis=dict(
#         title='No.of courses',
#         titlefont_size=16,
#         tickfont_size=14,
#     ))
 

In [929]:
# fig_semfreq = make_subplots(rows=3, cols=2,subplot_titles=("Semester 1", "Semester 2", "Semester 3", "Semester 4","Other Semester"))

# fig_semfreq.add_trace(
#     go.Bar(x=list(sem1['sum'].values), y=list(sem1['frequency'].values)),
#     row=1, col=1
# )

# fig_semfreq.add_trace(
#      go.Bar(x=list(sem2['sum'].values), y=list(sem2['frequency'].values)),
#     row=1, col=2
# )
# fig_semfreq.add_trace(
#      go.Bar(x=list(sem3['sum'].values), y=list(sem3['frequency'].values)),
#     row=2, col=1
# )
# fig_semfreq.add_trace(
#      go.Bar(x=list(sem4['sum'].values), y=list(sem4['frequency'].values)),
#     row=2, col=2
# )
# fig_semfreq.add_trace(
#      go.Bar(x=list(semo['sum'].values), y=list(semo['frequency'].values)),
#     row=3, col=1
# )

# fig_semfreq.update_layout(height=600, width=800, title_text="No.of Courses vs Participants")

In [930]:
sub_count=df_filtered.groupby('id').sub.value_counts(sort=True).to_frame()
sub_count_data=sub_count.rename(columns={sub_count.columns[0]: 'frequency'}).reset_index()

In [931]:
sub_count_data[sub_count_data['id']==30]

,id,sub,frequency
148,30,Cloud Web & Mobile,1
149,30,Distributed Systems,1
150,30,Electronic Communities and Social Networks,1
151,30,Information Engineering,1
152,30,Information Mining,1
153,30,Information Retrieval,1
154,30,Internet of Things- Protocols and System Software,1
155,30,Peer-to-Peer Systems,1


In [932]:
df_stud_complete=sub_count_data.groupby('id').frequency.sum().reset_index()

In [933]:
df_stud_complete[df_stud_complete['id']==30]

,id,frequency
29,30,8


### Count for number of students

In [934]:
count_credits=len(df_stud_complete[df_stud_complete['frequency']>=11])

In [935]:
fig_bar=px.bar(df_stud_complete,x='frequency',y='id',orientation='h')
fig_bar.show()

In [936]:
fig_sem = make_subplots(rows=3, cols=2,subplot_titles=("Semester 1", "Semester 2", "Semester 3", "Semester 4","Other Semesters"))

fig_sem.add_trace(go.Bar(x=sem11, y=sem12,name='Semester1'),
              row=1, col=1)

fig_sem.add_trace(go.Bar(x=sem21, y=sem22,name='Semester2'),
              row=1, col=2)
fig_sem.add_trace(go.Bar(x=sem31, y=sem32,name='Semester3'),
              row=2, col=1)
fig_sem.add_trace(go.Bar(x=sem41, y=sem42,name='Semester4'),
              row=2, col=2)
fig_sem.add_trace(go.Bar(x=semo1, y=semo2,name='Other Semester'),
              row=3, col=1)
fig_sem.update_layout(height=2500, width=600,
                  title_text="Semester Wise Courses Count",font=dict(size=7))

In [937]:
fig_grade = make_subplots(rows=3, cols=2,subplot_titles=("Grade A", "Grade B", "Grade C", "Grade D","Grade E","Grade F"))

fig_grade.add_trace(go.Bar(x=gradea1, y=gradea2,name='Grade A'),
              row=1, col=1)

fig_grade.add_trace(go.Bar(x=gradeb1, y=gradeb2,name='Grade B'),
              row=1, col=2)
fig_grade.add_trace(go.Bar(x=gradec1, y=gradec2,name='Grade C'),
              row=2, col=1)
fig_grade.add_trace(go.Bar(x=graded1, y=graded2,name='Grade D'),
              row=2, col=2)
fig_grade.add_trace(go.Bar(x=gradee1, y=gradee2,name='Grade E'),
              row=3, col=1)
fig_grade.add_trace(go.Bar(x=gradef1, y=gradef2,name='Grade F'),
              row=3, col=2)
fig_grade.update_layout(height=2500, width=900,
                  title_text="Grade Frequency")

In [999]:
external_stylesheets =['https://codepen.io/chriddyp/pen/bWLwgP.css',"https://cdnjs.cloudflare.com/ajax/libs/skeleton/2.0.4/skeleton.min.css",
                "https://fonts.googleapis.com/css?family=Raleway:400,400i,700,700i",
                "https://fonts.googleapis.com/css?family=Product+Sans:400,400i,700,700i"]

In [1031]:
app = dash.Dash(__name__,external_stylesheets=external_stylesheets)

In [1032]:
colors = {
    'background': '#6f32a8',
    'text': '#f2f542',
    'background-image': 'url("https://www.e-spincorp.com/wp-content/uploads/2019/03/Augmented-analytics-Capabilities-in-Business-Intelligence-1024x688.jpg")'
}

In [1033]:
image_filename = 'unilogo.png'
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

In [1034]:
card = html.Div(children=[
    html.H1('i-Recommend', style={
            'textAlign': 'center', 'margin': '48px 0', 'fontFamily': 'georgia','color':'#3269a8','background-color': 'lightblue'}),
    html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode())),
    dcc.Tabs(id="tabs", vertical=False,children=[
        dcc.Tab(label='Home', children=[
            html.H2('Welcome to the Course Advisory System',style={'textAlign': 'center', 'margin': '48px 0', 'fontFamily': 'georgia','color':'#3269a8'}),
            html.H4('Select the List of Courses you would like to attend:',style={'textAlign': 'left', 'margin': '48px 0', 'fontFamily': 'georgia','color':'#3269b8'}),
            html.Div(dcc.Input(id='input1',value='Enter keyword',type='hidden')),
            html.Div([dcc.Checklist(id='checklist1',
    options=[
        {'label': 'Advanced Image Synthesis', 'value': 'Advanced Image Synthesis'},
        {'label': 'Advanced Web Technologies', 'value': 'Advanced Web Technologies'},
        {'label': 'Cloud Web & Mobile', 'value': 'Cloud Web & Mobile'},
        {'label': 'Cognitive Robot Systems', 'value': 'Cognitive Robot Systems'},
        {'label': 'Computer Robot Vision', 'value': 'Computer Robot Vision'},
        {'label': 'Computer Graphics', 'value': 'Computer Graphics'},
        {'label': 'Gestaltung interaktiver Lehr-/Lern-Systeme', 'value': 'Gestaltung interaktiver Lehr-/Lern-Systeme'},
        {'label': 'Development of Safe and Secure Software', 'value': 'Development of Safe and Secure Software'},
        {'label': 'Digital Games Research', 'value': 'Digital Games Research'},
        {'label': 'Distributed Systems', 'value': 'Distributed Systems'},
        {'label': 'Electronic Communities and Social Networks', 'value': 'Electronic Communities and Social Networks'},
        {'label': 'Fault Diagnosis and Fault Tolerance in Technical Systems', 'value': 'Fault Diagnosis and Fault Tolerance in Technical Systems'},
        {'label': 'Formal Specification of Software Systems', 'value': 'Formal Specification of Software Systems'},
        {'label': 'Game Architecture and Design', 'value': 'Game Architecture and Design'},
        {'label': 'Information Engineering', 'value': 'Information Engineering'},
        {'label': 'Information Mining', 'value': 'Information Mining'},
        {'label': 'Information Retrieval', 'value': 'Information Retrieval'},
        {'label': 'Interaktive Systeme', 'value': 'Interaktive Systeme'},
        {'label': 'Internet of Things- Protocols and System Software', 'value': 'Internet of Things- Protocols and System Software'},
        {'label': 'Knowledge-Based Systems', 'value': 'Knowledge-Based Systems'},
        {'label': 'Learning Analytics', 'value': 'Learning Analytics'},
        {'label': 'Master Seminar Informatics', 'value': 'Master Seminar Informatics'},
        {'label': 'Modelling of Concurrent Systems', 'value': 'Modelling of Concurrent Systems'},
        {'label': 'Modelling-Analysis-Verification', 'value': 'Modelling-Analysis-Verification'},
        {'label': 'Natural-Language-based Human-Computer Interaction', 'value': 'Natural-Language-based Human-Computer Interaction'},
        {'label': 'Neurocomputing and Organic Computing', 'value': 'Neurocomputing and Organic Computing'},
        {'label': 'Pattern and Component based Software Development', 'value': 'Pattern and Component based Software Development'},
        {'label': 'Peer-to-Peer Systems', 'value': 'Peer-to-Peer Systems'},
        {'label': 'Recommender Systems', 'value': 'Recommender Systems'},
        {'label': 'Scientific Visualization', 'value': 'Scientific Visualization'},
        {'label': 'Test and Reliability of Digital Systems', 'value': 'Test and Reliability of Digital Systems'},
        
    ],
                                    #value=[]
  )
                     ],),
            html.Button('Suggest ME!', id='button1',style={'color':'#4287f5'}),
            
          #dt.DataTable(id='Table',columns=['Course','Suggested Courses'])
        ]),
        dcc.Tab(label='Graph1', children=[
            html.Div([
                html.H1("This is the content in tab 2"),
                dcc.Dropdown(id='dropdown_courses',
                options=[
                     {'label': 'Rating', 'value': 'Rating'},
        {'label': 'Grade', 'value': 'Grade'},
        {'label': 'Semester', 'value': 'Semester'}
                ]),
                html.Div(id='barchart',)
                
                #html.Div(className='card',children=[html.Img(src='unilogo.png',style={"width:100%"}),
                                                    #html.Div(children=[html.H4("Img1"),html.P("This is image1")])])
            ])
        ]),
        dcc.Tab(label='Graph2', children=[
            html.Div([
                html.H1("Generated Recommendations"),
                dcc.Store(id="value", storage_type="memory"),
                dcc.Graph(id='table1')
                #html.Table(id='table1')
                #dcc.Graph(id='content1',),
            ])
        ]),
        dcc.Tab(label='Graph3', children=[
            html.Div([
                html.H1("Some Charts"),
                #dcc.Store(id="value", storage_type="memory"),
                html.Div(dcc.Graph(id='scatter',figure=fig6)),
                html.Div(dcc.Graph(id='histo',figure=fig5)),
                html.Div(dcc.Graph(id='tot_bar',figure=fig_bar)),
                html.Cite('Based on the observations we can say that '+str(count_credits)+' students have completed the required 11 credits in 4 semesters')
            ])
        ]),
        dcc.Tab(label='Graph4', children=[
            html.Div([
                html.H1("Some Charts"),
                #dcc.Store(id="value", storage_type="memory"),
                html.Div(dcc.Graph(id='sankey',figure=fig_p)),
                
            ])
        ]),
        dcc.Tab(label='Algorithm based graphs', children=[
            html.Div([
                html.H1("Some Charts"),
                #dcc.Store(id="value", storage_type="memory"),
                html.Div(dcc.Graph(id='sup_conf',figure=fig3))
            ])
        ]),
        dcc.Tab(label='Itemsets', children=[
            html.Div([
                html.H1("Some Charts"),
                #dcc.Store(id="value", storage_type="memory"),
                html.Div(dcc.Graph(id='table_supconfi',figure=table_freqsets))
            ])
        ]),
        
       dcc.Tab(label='Graph5', children=[
            html.Div([
                html.H1("Course wise graphs"),
                 dcc.Dropdown(
        id='dropdown_courses1',
        options=[
         {'label': 'Advanced Image Synthesis', 'value': 'Advanced Image Synthesis'},
         {'label': 'Advanced Web Technologies', 'value': 'Advanced Web Technologies'},
         {'label': 'Cloud Web & Mobile', 'value': 'Cloud Web & Mobile'},
         {'label': 'Cognitive Robot Systems', 'value': 'Cognitive Robot Systems'},
         {'label': 'Computer Robot Vision', 'value': 'Computer Robot Vision'},
         {'label': 'Computer Graphics', 'value': 'Computer Graphics'},
         {'label': 'Gestaltung interaktiver Lehr-/Lern-Systeme', 'value': 'Gestaltung interaktiver Lehr-/Lern-Systeme'},
         {'label': 'Development of Safe and Secure Software', 'value': 'Development of Safe and Secure Software'},
         {'label': 'Digital Games Research', 'value': 'Digital Games Research'},
         {'label': 'Distributed Systems', 'value': 'Distributed Systems'},
         {'label': 'Electronic Communities and Social Networks', 'value': 'Electronic Communities and Social Networks'},
         {'label': 'Fault Diagnosis and Fault Tolerance in Technical Systems', 'value': 'Fault Diagnosis and Fault Tolerance in Technical Systems'},
         {'label': 'Formal Specification of Software Systems', 'value': 'Formal Specification of Software Systems'},
         {'label': 'Game Architecture and Design', 'value': 'Game Architecture and Design'},
         {'label': 'Information Engineering', 'value': 'Information Engineering'},
         {'label': 'Information Mining', 'value': 'Information Mining'},
         {'label': 'Information Retrieval', 'value': 'Information Retrieval'},
         {'label': 'Interaktive Systeme', 'value': 'Interaktive Systeme'},
         {'label': 'Internet of Things- Protocols and System Software', 'value': 'Internet of Things- Protocols and System Software'},
         {'label': 'Knowledge-Based Systems', 'value': 'Knowledge-Based Systems'},
         {'label': 'Learning Analytics', 'value': 'Learning Analytics'},
         {'label': 'Master Seminar Informatics', 'value': 'Master Seminar Informatics'},
         {'label': 'Modelling of Concurrent Systems', 'value': 'Modelling of Concurrent Systems'},
         {'label': 'Modelling-Analysis-Verification', 'value': 'Modelling-Analysis-Verification'},
         {'label': 'Natural-Language-based Human-Computer Interaction', 'value': 'Natural-Language-based Human-Computer Interaction'},
         {'label': 'Neurocomputing and Organic Computing', 'value': 'Neurocomputing and Organic Computing'},
         {'label': 'Pattern and Component based Software Development', 'value': 'Pattern and Component based Software Development'},
         {'label': 'Peer-to-Peer Systems', 'value': 'Peer-to-Peer Systems'},
         {'label': 'Recommender Systems', 'value': 'Recommender Systems'},
         {'label': 'Scientific Visualization', 'value': 'Scientific Visualization'},
         {'label': 'Test and Reliability of Digital Systems', 'value': 'Test and Reliability of Digital Systems'}
        ]
    ),
                html.Div(id='fig_subplot',)
                #dcc.Store(id="value", storage_type="memory"),
            ])
        ])
    ],
        style={
        'fontFamily': 'system-ui',
        #'text': '#f2f542',
        'background': '#ebeb34'
            
    },
        content_style={
        'borderLeft': '1px solid #d6d6d6',
        'borderRight': '1px solid #d6d6d6',
        'borderBottom': '1px solid #d6d6d6',
        'padding': '44px'
    },
        parent_style={
        'maxWidth': '1000px',
        'margin': '0 auto',
        'background': '#ffffff',
        'color':'#000000',
        'font-size': 'large'
    }
    ),
])


In [1035]:
import os
os.getcwd()

'C:\\Users\\GOLLAKOTA'

In [1036]:
app.layout=html.Div([card])

In [1037]:
#fig_sub.add_trace(go.Bar(x=list(df_ratings['sub'].values),y=list(df_ratings['rating'].values)),row=1,col=1)

In [1038]:
@app.callback(Output('barchart', 'children'),
              [Input('dropdown_courses', 'value')])
def update_figure(value):
    if value is None:
        return html.Label('No value has been selected.Please select a value')
    elif value=='Rating':
        return html.Div([
            html.Br(),
                dcc.Graph(
                    id='bar chart',
                    figure=fig11
                ),
        ])
    elif value=='Grade':
        return html.Div([
                dcc.Graph(
                    id='bar chart',
                    figure=fig_grade,
                ),
            html.Cite('Based on the observations we conclude that lot of students get good grade in '+easy_sub+
                          ' and majority of the students find it difficult to get good grades in '+diff_sub)
        ])
    elif value=='Semester':
        return html.Div(
                dcc.Graph(
                    id='bar chart',
                    figure=fig_sem,
                ),
        )

In [1039]:
@app.callback(Output('fig_subplot', 'children'),
              [Input('dropdown_courses1', 'value')])
def update_figure(value):
    if value is None:
        return html.Label('No value has been selected.Please select a value')
    else:
        fig_subplots=make_subplots(rows=1, cols=3,subplot_titles=("Rating", "Grades", "Semester"))
        fig_subplots.add_trace(go.Bar(x=list(get_rating_frequency(value)['rating'].values),y=list(get_rating_frequency(value)['frequency'].values),name='Rating'),row=1,col=1)
        fig_subplots.add_trace(go.Bar(x=list(get_grade_frequency(value)['grade'].values),y=list(get_grade_frequency(value)['frequency'].values),name='Grades'),row=1,col=2)
        fig_subplots.add_trace(go.Bar(x=list(get_semester_frequency(value)['semester'].values),y=list(get_semester_frequency(value)['frequency'].values),name='Semester'),row=1,col=3)
        fig_subplots.update_layout(height=600,width=900)

        return html.Div(
            dcc.Graph(
            id='fig subplot',
            figure=fig_subplots))

In [1040]:
df_rec['sub'].values

array(['Advanced Image Synthesis', 'Advanced Web Technologies',
       'Development of Safe and Secure Software', 'Distributed Systems',
       'Information Retrieval', 'Learning Analytics',
       'Peer-to-Peer Systems', 'Advanced Web Technologies',
       'Cloud Web & Mobile', 'Cognitive Robot Systems',
       'Development of Safe and Secure Software', 'Distributed Systems',
       'Electronic Communities and Social Networks',
       'Gestaltung interaktiver Lehr-/Lern-Systeme',
       'Information Engineering', 'Information Retrieval',
       'Learning Analytics', 'Peer-to-Peer Systems',
       'Distributed Systems',
       'Electronic Communities and Social Networks',
       'Formal Specification of Software Systems',
       'Gestaltung interaktiver Lehr-/Lern-Systeme',
       'Information Engineering', 'Information Mining',
       'Information Retrieval', 'Interaktive Systeme',
       'Internet of Things- Protocols and System Software',
       'Learning Analytics', 'Recommender Sy

In [1041]:

@app.callback(Output("table1", "figure"), [Input("button1", "n_clicks")],[State("checklist1", "value")])
def update_output(rows,value):
    list_updated=[]
    if rows is not None:
       # Here, I could get DataFrames
        for val in value:
            if val not in list(df_rec['sub'].values):
                print('No recommendations') 
            else:
                list_updated.append(val)
        l2=recommend_courses(list_updated)
        
        #print(set(l3))
        
        df1 = pd.DataFrame({'Courses' : list_updated,
                                'Recommended_Courses' : l2 }, 
                                columns=['Courses','Recommended_Courses'])
        print(df1.head(5))
    
      # But, have no ideas how to return dash_table.Databales instead ob go.Table
        
        #table = ff.create_table(df1,height_constant=30)
        #table.layout.width=250
        figup=go.Figure(data=[go.Table(columnwidth = [700,3000],
        header=dict(values=list(df1.columns),
                fill_color='purple',
                align='left',
                height=40,
               font=dict(color='white', size=18,family='sans-serif')),
    cells=dict(values=[df1.Courses, df1.Recommended_Courses],
               fill_color='lavender',
               height=60,
               align='left',font=dict(size=14,family='sans-serif')))
],)
        return figup
        


In [1042]:
if __name__ == '__main__':
    app.run_server(debug=False)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off
